<a href="https://colab.research.google.com/github/Lotfullah21/PyLibraries/blob/main/Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import keras
import tensorflow as tf

In [3]:
def generate(batch_size,n_steps):
  freq1,freq2,offset1,offset2 = np.random.rand(4,batch_size,1)
  time = np.linspace(0,1,n_steps)
  series = 0.5*np.sin((time-offset1)*(freq1*10+10))
  series += 0.2*np.sin((time-offset2)*(freq2*20+20))
  series += 0.1*(np.random.rand(batch_size,n_steps)-0.5)
  return  series[...,np.newaxis].astype(np.float32)

In [7]:
num_steps = 90
series = generate(10000,num_steps+1)
x_train, y_train = series[:7000,:num_steps],series[:7000,-1]
x_valid,y_valid = series[7000:9000,:num_steps],series[7000:9000,-1]
x_test,y_test = series[9000:,:num_steps],series[9000:-1]



the training has the shape of [7000,90,1] and the target has the shape of [7000,1], becaues we want to forecast a single value per time step.

In [11]:
y_pred  = x_valid[:,-1]
np.mean(keras.losses.mean_squared_error(y_valid,y_pred))

0.0076056398

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[90,1]),
    keras.layers.Dense(1)
])

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.mean_squared_error,
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

In [23]:
# # model.fit(x_train,y_train,validation_data = ([x_valid,y_valid])
# model.fit(x_train,y_train,validation_data = ([x_valid,y_valid]),epochs=9)

In [24]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1,input_shape=[None,1])
])
# single layer, single neuron, RNN can process any number of time steps, that is why we putted none,

 single layer, single neuron, RNN can process any number of time steps, that is why we putted none, and by default the RNN uses the tanh as activation function, by default the R layers in keras returns final output, if we want to do it one output per time step , then use *return_sequence = True*

## Deep RNN
just, stack recurrent layers

In [27]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences = True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences = True),
    keras.layers.SimpleRNN(1)
])

make sure you add the return_sequences = True for all layers, other wise it will need feed the last time steps to the next layer,,

 instead of *keras.layers.SimpleRNN(1)* this , we can use the Dense layer and that is for two reasons, one is it will run quite faster and the 2nd reason is that it will allow us to pick the activation of our own choice.

In [28]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences = True,input_shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences = True),
    keras.layers.Dense(1)
])

## ForeCasting Several Times ahead

to predict 10 steps ahead , just change the target to 10 steps ahead, 

In [29]:
series = generate(1,num_steps+10)
x_new,y_new = series[:,:num_steps],series[:,num_steps:]
x = x_new
for steps_ahead in range(10):
  y_pred_one = model.predict(x[:,steps_ahead:])[:,np.newaxis,:]
  x = np.concatenate([x,y_pred_one],axis = 1)
y_pred = x[:,num_steps:]  

ValueError: ignored

In [34]:
num_steps = 50
series = generate(10000,num_steps+10)
x_train, y_train = series[:7000,:num_steps],series[:7000,-10:,0]
x_valid,y_valid = series[7000:9000,:num_steps],series[7000:9000,-10:,0]
x_test,y_test = series[9000:,:num_steps],series[9000:-10:,0]


In [38]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences = True,input_shape=[50,1]),
    keras.layers.SimpleRNN(20,return_sequences = True),
    keras.layers.Dense(10)
])

In [40]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.mean_squared_error,
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

In [41]:
# model.fit(x_train,y_train,epochs = 10)

Epoch 1/10


ValueError: ignored